## LDC Lens Distortion Correction

In [ ]:
# pip3 install opencv-python
import cv2
import numpy as np
%matplotlib notebook
from matplotlib import pyplot as plt

In [ ]:
def extractFrames(path_to_video, path_to_frames): 
    vidObj = cv2.VideoCapture(path_to_video) 
    count, success = 0, 1
    while success: 
        success, image = vidObj.read() 
        cv2.imwrite(path_to_frames + "/frame%d.jpg" % count, image) 
        count += 1
    print(f"Count of frames: {count}")

In [ ]:
path_to_data = "../../mydata/evidences/5d952e0f-51fe-e1f2-a0aa-551938968072" 
video_file = path_to_data + "/video.mp4"
path_to_frames = path_to_data + "/frames"
extractFrames(video_file, path_to_frames) 

#### Example 1
https://hackaday.io/project/12384-autofan-automated-control-of-air-flow/log/41862-correcting-for-lens-distortions

https://docs.opencv.org/2.4/doc/tutorials/calib3d/camera_calibration/camera_calibration.html

This so-called barrel distortion results in the fact that the representation of distance relations in the real world is not the same as in the camera image -- i.e. distance relations in the camera image are non-linear.

In [ ]:
def lens_distortion_correction(K, d, img_file):
    """
    K - camera matrix
    d - distortion coefficients
    img_file - path to img
    
    return (img, newimg)
    """

    img = cv2.imread(img_file)
    h, w = img.shape[:2]

    # Generate new camera matrix from parameters
    newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(K, d, (w,h), 0)

    # Generate look-up tables for remapping the camera image
    mapx, mapy = cv2.initUndistortRectifyMap(K, d, None, newcameramatrix, (w, h), 5)

    # Remap the original image to a new image
    newimg = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
    
    return img, newimg

def display_images(img, newimg):
    fig, (oldimg_ax, newimg_ax) = plt.subplots(1, 2)
    oldimg_ax.imshow(img)
    oldimg_ax.set_title('Original image')
    newimg_ax.imshow(newimg)
    newimg_ax.set_title('Unwarped image')
    plt.show()

In [ ]:
# Define camera matrix K
K = np.array([[673.9683892, 0., 343.68638231],
              [0., 676.08466459, 245.31865398],
              [0., 0., 1.]])

# Define distortion coefficients d
d = np.array([5.44787247e-02, 1.23043244e-01, -4.52559581e-04, 5.47011732e-03, -6.83110234e-01])

img_file = path_to_frames + "/frame30.jpg"
img, newimg = lens_distortion_correction(K, d, img_file)
display_images(img, newimg)

In [ ]:
# python calibrate.py "frames/frame30.jpg"

In [ ]:
cv2.__file__

## Camera calibration
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html

In [ ]:
import os
import numpy as np
import cv2

In [ ]:
dir_data = '/Users/silinskiy/mydata/chess_board/5de94dd1-963e-934c-5be5-551938968072'
dir_inp = os.path.join(dir_data, 'frames')
dir_out = os.path.join(dir_data, 'frames_corrected')
dir_corners = None # os.path.join(dir_data, 'frames_corners')

for d in [dir_out, dir_corners]:
    if d and not os.path.exists(d):
        os.makedirs(d)

img = cv2.imread(os.path.join(dir_inp, os.listdir(dir_inp)[0]))
h,  w = img.shape[:2]

In [ ]:
def find_the_chess_board_corners(img, criteria, vertical_points, horizont_points):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (vertical_points, horizont_points), None)
    return cv2.cornerSubPix(gray,corners,(11,11),(-1,-1), criteria) if ret else None

In [ ]:
horizont_points = 7
vertical_points = 7

frame_number_min = 0
frame_number_max = 1244

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((horizont_points*vertical_points,3), np.float32)
objp[:,:2] = np.mgrid[0:vertical_points,0:horizont_points].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

for n in range(frame_number_min,frame_number_max+1):
    file = f'frame{n}.jpg'
    img = cv2.imread(os.path.join(dir_inp, file))
    
    corners = find_the_chess_board_corners(img, criteria, vertical_points, horizont_points)
    if corners is not None:
        print(file)
        imgpoints.append(corners)
        objpoints.append(objp)
        
        # Draw corners and save image to file
        if dir_corners:
            img2 = cv2.drawChessboardCorners(img, (vertical_points,horizont_points), corners, True)
            cv2.imwrite(os.path.join(dir_corners, file), img2)

### Calibration
camera matrix, distortion coefficients, rotation and translation vectors etc.

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (w, h), None, None)

### Undistortion


In [ ]:
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

#### 1. Using cv2.undistort()

In [ ]:
# undistort
file = 'frame555.jpg'
img = cv2.imread(os.path.join(dir_inp, file))
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite(os.path.join(dir_out, file), dst)

#### 2. Using remapping

In [ ]:
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

### Re-projection Error

In [ ]:
mean_error = 0
for i in xrange(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print "total error: ", mean_error/len(objpoints)

## Video calibration

In [ ]:
import os
import numpy as np
import cv2

def calc_calibration_matrix(path_to_video, max_image_count = 50, horizont_points = 7, vertical_points = 7):
    videoInp = cv2.VideoCapture(path_to_video)
    w = int(videoInp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(videoInp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f'video: width = {w} x height = {h}')
    
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((horizont_points*vertical_points,3), np.float32)
    objp[:,:2] = np.mgrid[0:vertical_points,0:horizont_points].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    
    count = 0
    ok, img = videoInp.read()
#     h,  w = img.shape[:2]
    while ok and count < max_image_count:
        corners = find_the_chess_board_corners(img, criteria, vertical_points, horizont_points)
        if corners is not None:
            objpoints.append(objp)
            imgpoints.append(corners)
            count += 1
        ok, img = videoInp.read()
    videoInp.release()
    if count > 0:
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (w, h), None, None)
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
        return (mtx, dist, newcameramtx)

def calibrate_video(path_to_video, path_to_otput_video, mtx, dist, newcameramtx):
    videoInp = cv2.VideoCapture(path_to_video)
    w = int(videoInp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(videoInp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30
    videoOut = cv2.VideoWriter(path_to_otput_video, cv2.VideoWriter_fourcc(*'H264'), fps, (w, h))
    
    ok, img = videoInp.read()
    h,  w = img.shape[:2]
    while ok:
        img2 = cv2.undistort(img, mtx, dist, None, newcameramtx)
        videoOut.write(img2)
        ok, img = videoInp.read()
    videoInp.release()
    videoOut.release()

In [ ]:
path_to_video = os.path.join(dir_data, 'video.mp4')
mtx, dist, newcameramtx = calc_calibration_matrix(path_to_video, max_image_count = 20)
path_to_otput_video = os.path.join(dir_data, 'video_calibrated.mp4')
#calibrate_video(path_to_video, path_to_otput_video, mtx, dist, newcameramtx)

In [ ]:
import sys
sys.path.insert(0, "/Users/silinskiy/axon_git/monitor-detection-tracking/mdt/detector/")
from transform import Transform
T = Transform()